In [ ]:
import json
from datetime import datetime, timedelta, date
import time
import pandas as pd
import openpyxl
import numpy as np

#--- STEP0 --- : Init
#-> Repository
input_folder = "../data_input/"
output_folder = "../data_output/"
param_folder = "../../_parameters/"

#-> Date
now = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
today = date.today()
yesterday = (today - timedelta(days = 1))
b_yesterday = (today - timedelta(days = 2))

#-> Get credentials info
credentials = param_folder + 'mongodb.json'
with open(credentials) as f:
    dbCon = json.load(f)

mongo_host = dbCon["host"]
mongo_port = dbCon["port"]
mongo_username = dbCon["username"]
mongo_password = dbCon["password"]

#-> Bob connect
MongoDB = bob.mongo.connect(mongo_host,mongo_port,mongo_username,mongo_password)
db_src = 'covid-19-dev'
db_app = 'app-wsr-dev'